In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [ ]:
spark = SparkSession.builder.appName('apriori') \
                    .config("spark.executor.memory") \
                    .getOrCreate()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving store_data.csv to store_data (1).csv


In [ ]:
import pandas as pd
import io

df = spark.read.csv("store_data.csv")
df
# df.printSchema() 

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string]

### Hàm generate_unique_item_set()
Giá trị trả về: Khung dữ liệu với các mục duy nhất (không lặp lại) và các giá trị rỗng bị xóa khỏi dataFrame

- df.na cung cấp tất cả các giá trị null; tất cả các giá trị null phải được loại bỏ
- rdd chuyển đổi DataFrame thành RDD
- remove_duplicate_columns phải được áp dụng cho các phần tử của RDD để mỗi mục trong df là một phần tử riêng biệt
- Hàm 'remove_duplicate_columns' được áp dụng cho từng phần tử của RDD, mọi hàng của khung dữ liệu phải được chuyển vào đó.

In [ ]:
# trả về danh sách các unique_item trong tập dữ liệu
def generate_unique_item_set(df):
    # empty dataframe
    total_item_set_df = None
    
    for col_index in range(20):
        #  Lấy các phần tử của từng cột riêng lẻ
        _c_df = df.select("_c" + str(col_index))

        if total_item_set_df is None:
            total_item_set_df = _c_df
        # Thêm các phần tử _c_df từ mỗi cột vào total_item_set_df
        else:
            total_item_set_df = total_item_set_df.union(_c_df)
            
    return total_item_set_df.select("_c0").na.drop().rdd.flatMap(remove_duplicate_columns).distinct().toDF()

### Hàm remove_duplicate_columns()
- Loại bỏ các cột có giá trị trùng lắp

- Trả về mảng đã remove các cột trùng lắp

In [ ]:
def remove_duplicate_columns(x):
    col_len = len(x)
    # RDD rỗng chứa tập hợp các values
    columns = set()
    # Remove khoảng trắng 
    # khỏi các phần tử và thêm các phần tử vào cột từ RDD 'x'
    for col in range(col_len):
        x_col = str(x[col]).strip() #  xóa khoảng trắng cuối string
        columns.add(x_col)
    # Nếu ko có duplicate columns thì return rỗng
    if len(columns) < col_len:
        return []
    
    return [(tuple(sorted(columns)))]

In [ ]:
item_sets = generate_unique_item_set(df)
item_sets.show()

+--------------------+
|                  _1|
+--------------------+
|   whole wheat pasta|
|           asparagus|
|            pancakes|
|         blueberries|
|            zucchini|
|              shrimp|
|             burgers|
|           spaghetti|
|         french wine|
|       strong cheese|
|extra dark chocolate|
|              melons|
|               cream|
|   frozen vegetables|
|           meatballs|
|          energy bar|
|            escalope|
|        energy drink|
|                mint|
|      vegetables mix|
+--------------------+
only showing top 20 rows



In [ ]:
# Số lượng unique items trong dataset
item_sets.count()

119

### Hàm get_freq_item_sets()
- Tạo các itemsets đã lọc từ dataFrame được cung cấp dựa trên min support. 
- Lọc các tập hợp ứng viên theo min support lấy từ hàm Apriori(). 
- Kq trả về một mảng để tạo ra đầu ra cuối cùng của hàm Apriori().

In [ ]:
def get_freq_item_sets(total_records, candidate_sets_shared, transaction_df_rdd, min_support):    
    filtered_item_set = transaction_df_rdd.flatMap(lambda x: filter_and_map_transaction(x, candidate_sets_shared)).reduceByKey(lambda a, b: a+b).map(lambda x: (x[0], x[1]/total_records)).filter(lambda x: x[1]>=min_support)
    return filtered_item_set

### Hàm filter_and_map_transaction()

So sánh từng items với tập hợp ứng viên. Nếu item thuộc giao dịch, item tương ứng trong nhóm ứng viên sẽ được ánh xạ với giá trị 1, ngược lại là giá trị 0

In [ ]:
def filter_and_map_transaction(x, candidate_set_shared):
    
    c_k = []
    
    rows = len(candidate_set_shared.value)
    cols = len(candidate_set_shared.value[0])
    
    # Checking each transaction
    for row_i in range(rows):
        item_set = set()
        for col_i in range(cols):
            item_set.add(candidate_set_shared.value[row_i][col_i])
        
        # Map the condition with the correct value (1/0)
        if item_set.issubset(set(x)):
            c_k.append((candidate_set_shared.value[row_i], 1))
        else:
            c_k.append((candidate_set_shared.value[row_i], 0))
    return c_k

### Hàm is_freq_item_set_not_empty()
-  Kiểm tra xem tập freq_item_sets có bất kỳ giao dịch nào hay không
- Điều kiện đầu tiên được xác định để kiểm tra xem có các phần tử trong tập phổ biến hay không.
Có thể có trường hợp các phần tử có mặt nhưng tất cả chúng đều không có.
- Điều kiện thứ hai kiểm tra nếu tất cả các phần tử của tập phổ biến không phải là không có.

In [ ]:
def is_freq_item_set_not_empty(freq_item_sets):
    return (freq_item_sets.count() > 0) and (freq_item_sets is not None)

### Hàm get_all_possible_candidate_sets()
Tạo các tập phổ biến có thứ tự cao hơn từ đơn hàng đầu tiên. Giá trị 'k + 1' biểu thị số thứ tự và hàm tạo các tập phổ biến có thứ tự 'k + 1' bằng cách hợp nhất hai tập mục đầu vào có thứ tự k & 1 tương ứng.

In [ ]:
def get_all_possible_candidate_sets(candidate_item_sets_k, candidate_item_sets_0):
    
    # Convert các phần tử của candidate_item_sets_k từ list format vào list tuple
    candidate_item_sets_k = candidate_item_sets_k.map(lambda x: tuple(x[0])).toDF()
    # toDF(): rdd -> df
    
    # Return tập thứ k+1
    return candidate_item_sets_k.crossJoin(candidate_item_sets_0).rdd.flatMap(remove_duplicate_columns).distinct()
    # crossJoin(): kết hợp một phần tử của một dataFrame với tất cả các phần tử của dataFrame khác


- item_sets: DataFrame có tất cả các mục có trong các giao dịch

- transaction_df_rdd: RDD dataFrame đã xử lý bên trên
    
- min_support: Giá trị min sup


In [ ]:
from pyspark.sql.types import StructType, ArrayType, StructField, DoubleType, StringType

def apriori(item_sets, transaction_df_rdd, min_support):
    
    # Đếm tổng số values  trong mỗi colunms trong dataframe
    total_records = transaction_df_rdd.count()
    #Khởi tạo 1 mảng trống dùng để lưu trữ các frequent itemset
    freq_item_sets_all_orders = []
    # candidate_sets_order_1
    # là danh sách mặt hàng hoàn chỉnh từ các giao dịch
    # broadcast được sử dụng để gửi một values trong item_sest tới tất cả các cluster

    candidate_sets_order_1 = spark.sparkContext.broadcast(item_sets.collect())
    
    # Gọi hàm get_freq_items_sests để tạo tập hợp frequent_item_sets_order_1
    # Sau đó gán kết quả vào freq_item_sets_all_orders

    frequent_item_sets_order_1 = get_freq_item_sets(total_records, candidate_sets_order_1, transaction_df_rdd, min_support)
    freq_item_sets_all_orders.append(frequent_item_sets_order_1)
    # Chuyển đổi các phần tử rdd của  frequent_item_sets_order_1' thành dataFrame 
    frequent_item_sets_order_1_df = frequent_item_sets_order_1.map(lambda x:tuple(x[0])).toDF()
    
    k = 0
    # Vòng lặp sẽ chạy cho đến khi có thể tạo các items_set
    while is_freq_item_set_not_empty(freq_item_sets_all_orders[k]):
        # Tạo candidate sets
        # Lọc  candidate sets  để nhận nhóm item_sest thường gặp theo thứ tự 'k + 1'
        # Thêm danh sách 'freq_item_sets_all_orders' với frequent itemset k + 1

        current_candidate_sets = get_all_possible_candidate_sets(freq_item_sets_all_orders[k], frequent_item_sets_order_1_df)
        current_candidate_sets = spark.sparkContext.broadcast(current_candidate_sets.collect()) 
        current_frequent_item_sets = get_freq_item_sets(total_records, current_candidate_sets, transaction_df_rdd, min_support)
        # # freq_item_sets_all_orders là danh sách các RDD trong đó phần tử k lưu trữ tập hợp mục thường xuyên của đơn hàng k + 1
        freq_item_sets_all_orders.append(current_frequent_item_sets)
        k += 1
    
    
    return freq_item_sets_all_orders

In [ ]:
df_rdd = df.rdd
freq_item_sets_all_orders = apriori(item_sets, df_rdd, 0.01)

print(freq_item_sets_all_orders)

[PythonRDD[600] at RDD at PythonRDD.scala:53, PythonRDD[601] at RDD at PythonRDD.scala:53, PythonRDD[602] at RDD at PythonRDD.scala:53, PythonRDD[603] at RDD at PythonRDD.scala:53]


In [ ]:
freq_item_sets_all_orders[0].take(20)

[(Row(_1='whole wheat pasta'), 0.029462738301559793),
 (Row(_1='pancakes'), 0.09505399280095987),
 (Row(_1='shrimp'), 0.07145713904812692),
 (Row(_1='burgers'), 0.0871883748833489),
 (Row(_1='spaghetti'), 0.17411011865084655),
 (Row(_1='french wine'), 0.022530329289428077),
 (Row(_1='extra dark chocolate'), 0.011998400213304892),
 (Row(_1='melons'), 0.011998400213304892),
 (Row(_1='frozen vegetables'), 0.09532062391681109),
 (Row(_1='meatballs'), 0.020930542594320756),
 (Row(_1='energy bar'), 0.027063058258898813),
 (Row(_1='escalope'), 0.0793227569657379),
 (Row(_1='energy drink'), 0.026663111585121985),
 (Row(_1='mint'), 0.0174643380882549),
 (Row(_1='vegetables mix'), 0.025729902679642713),
 (Row(_1='body spray'), 0.011465137981602452),
 (Row(_1='chocolate'), 0.1638448206905746),
 (Row(_1='butter'), 0.030129316091187842),
 (Row(_1='eggplant'), 0.013198240234635382),
 (Row(_1='fresh tuna'), 0.022263698173576856)]

In [ ]:
freq_item_sets_all_orders[1].take(20)

[(('pancakes', 'shrimp'), 0.010531929076123183),
 (('burgers', 'pancakes'), 0.010531929076123183),
 (('pancakes', 'spaghetti'), 0.025196640447940274),
 (('frozen vegetables', 'pancakes'), 0.013464871350486601),
 (('chocolate', 'pancakes'), 0.01986401813091588),
 (('green tea', 'pancakes'), 0.01639781362485002),
 (('milk', 'pancakes'), 0.01653112918277563),
 (('eggs', 'pancakes'), 0.021730435941874418),
 (('mineral water', 'pancakes'), 0.03372883615517931),
 (('french fries', 'pancakes'), 0.020130649246767097),
 (('ground beef', 'pancakes'), 0.014531395813891481),
 (('cake', 'pancakes'), 0.011865084655379284),
 (('olive oil', 'pancakes'), 0.010798560191974404),
 (('shrimp', 'spaghetti'), 0.021197173710171976),
 (('frozen vegetables', 'shrimp'), 0.01666444474070124),
 (('chocolate', 'shrimp'), 0.017997600319957337),
 (('shrimp', 'tomatoes'), 0.011198506865751233),
 (('green tea', 'shrimp'), 0.011465137981602452),
 (('milk', 'shrimp'), 0.01759765364618051),
 (('eggs', 'shrimp'), 0.0141314

In [ ]:
freq_item_sets_all_orders[2].take(20)

[(('mineral water', 'pancakes', 'spaghetti'), 0.011465137981602452),
 (('frozen vegetables', 'mineral water', 'spaghetti'), 0.011998400213304892),
 (('chocolate', 'milk', 'spaghetti'), 0.010931875749900012),
 (('chocolate', 'eggs', 'spaghetti'), 0.010531929076123183),
 (('chocolate', 'mineral water', 'spaghetti'), 0.01586455139314758),
 (('milk', 'mineral water', 'spaghetti'), 0.01573123583522197),
 (('eggs', 'mineral water', 'spaghetti'), 0.014264764698040262),
 (('french fries', 'mineral water', 'spaghetti'), 0.010131982402346354),
 (('ground beef', 'mineral water', 'spaghetti'), 0.017064391414478068),
 (('mineral water', 'olive oil', 'spaghetti'), 0.010265297960271964),
 (('frozen vegetables', 'milk', 'mineral water'), 0.011065191307825623),
 (('chocolate', 'milk', 'mineral water'), 0.013998133582189041),
 (('chocolate', 'eggs', 'mineral water'), 0.013464871350486601),
 (('chocolate', 'ground beef', 'mineral water'), 0.010931875749900012),
 (('eggs', 'milk', 'mineral water'), 0.0130

In [ ]:
# Tạo tất cả các tập hợp con có thể có từ một tập hợp

def powerset(s):
    slist = list(s)
    result = [[]]
    for x in slist:
        
        result.extend([subset + [x] for subset in result])
        
    return [item_set for item_set in result if (len(item_set) > 0 and len(item_set) < len(slist))]

#### Hàm generate_rules_with_confidence()
- Tạo rules với confidence   
- x: freq_item_sets_all_orders; danh sách chứa các frequent item set cùng với confidence score
- k: thứ tự của frequent item set  
- freq_item_sets_map_all_orders_shared: broadcasted map (cặp key-value ) của frequent item sets: 
    - key là subset
    - value là corresponding support  
- min_confidence: Giá trị min confidence 


In [ ]:
from pyspark.sql import Row

def generate_rules_with_confidence(x, k, freq_item_sets_map_all_orders_shared, min_confidence):
 
    # Trích xuất Cột đầu tiên của RDD
    freq_item_set = set(x[0])
    
    # all_subsets lưu trữ tất cả các tập con từ freq_item_set
    all_subsets = powerset(freq_item_set)
    

    rules = []
    
    # Convert các broadcasted values theo định dạng của list tuple
    freq_item_set_support = freq_item_sets_map_all_orders_shared[k].value[tuple(sorted(freq_item_set))]

    
    candidate_set_key = ''
    subset_k = 0
    for subset in all_subsets:
        antecedent = subset
        
        # Consequent được tạo ra bằng cách xóa antecedent khỏi tập frequent item set
        consequent = freq_item_set - set(antecedent)
    
        # Kiểm tra TH có một phần tử duy nhất trong antecedent
        if (len(consequent) == 1):
            single_item = consequent.pop()
            candidate_set_key = Row(_1=single_item)
            subset_k = 1
            consequent.add(single_item)
        else:
            candidate_set_key = tuple(sorted(consequent))
            subset_k = len(consequent) 

        # support value cho consequent
        # value.get lấy values từ cặp key-value
        consequent_support = freq_item_sets_map_all_orders_shared[subset_k-1].value.get(candidate_set_key)
        
        if (consequent_support is not None):
            
            # Tính giá trị confidence value cho rules
            confidence = freq_item_set_support/consequent_support
        
            # Thêm rules nếu giá trị confidence lớn hơn ngưỡng confidence
            if (confidence >= min_confidence):
                rules.append((list(antecedent), list(consequent), confidence))      

    return rules

#### Hàm generate_association_rules_for_k_order
- Tạo ra các quy tắc kết hợp cho các tập phổ biến có thứ tự k

- k: thứ tự của frequent item set  
- freq_item_sets_map_all_orders_shared: broadcasted map (cặp key-value ) của frequent item sets: 
    - key là subset
    - value là corresponding support  
- min_confidence: Giá trị min confidence  

In [ ]:
def generate_association_rules_for_k_order(k, freq_item_sets_order_k, \
                                           freq_item_sets_map_all_orders_shared, min_confidence):
    # Hàm generate_rules_with_confidence là một flat map tính frequent trong itemsets theo thứ tự k
    return freq_item_sets_order_k.flatMap(lambda x: generate_rules_with_confidence(x, \
                                  k,freq_item_sets_map_all_orders_shared, \
                                  min_confidence)).toDF(('Antecedent', 'Consequent', 'Confidence'))
                                  # Chuyển RDD thành Dataframe và thay đổi tên cột

#### Hàm generate_association_rules

- Đệ quy trên tất cả các frequent itemsets của các transaction khác nhau và tạo ra một tổ hợp các quy tắc kết hợp
- freq_item_sets_all_orders: danh sách tất cả frequent itemsets (
- min_confidence

In [ ]:
def generate_association_rules(freq_item_sets_all_orders, min_confidence):
    # Gửi broadcast
    freq_item_sets_map_all_orders_shared = [spark.sparkContext.broadcast(freq_item_sets.collectAsMap()) \
                                                                     for freq_item_sets in freq_item_sets_all_orders]
    
    l = len(freq_item_sets_map_all_orders_shared)
    
    # tất cả hàm được xác định ở trên để tạo các quy tắc từ tập phổ biến thường xuyên có thứ tự 2
    association_rules_df = generate_association_rules_for_k_order(1, freq_item_sets_all_orders[1], \
                                                                  freq_item_sets_map_all_orders_shared, \
                                                                  min_confidence)
    
    for k in range(2, l):
        if is_freq_item_set_not_empty(freq_item_sets_all_orders[k]):
            # Kết hợp tất cả association rules cho mỗi frequent itemsets
            
            association_rules_df = association_rules_df.union(generate_association_rules_for_k_order(k, \
                                                        freq_item_sets_all_orders[k], \
                                                        freq_item_sets_map_all_orders_shared, min_confidence))
    return association_rules_df

In [ ]:
association_rules = generate_association_rules(freq_item_sets_all_orders, 0.099)

In [ ]:
association_rules.show(200)

+-------------------+-------------------+-------------------+
|         Antecedent|         Consequent|         Confidence|
+-------------------+-------------------+-------------------+
|         [pancakes]|           [shrimp]|0.14738805970149252|
|           [shrimp]|         [pancakes]|0.11079943899018233|
|          [burgers]|         [pancakes]|0.11079943899018233|
|         [pancakes]|          [burgers]|0.12079510703363913|
|        [spaghetti]|         [pancakes]| 0.2650771388499299|
|         [pancakes]|        [spaghetti]| 0.1447166921898928|
|         [pancakes]|[frozen vegetables]|0.14125874125874124|
|[frozen vegetables]|         [pancakes]|0.14165497896213183|
|         [pancakes]|        [chocolate]|0.12123677786818551|
|        [chocolate]|         [pancakes]|0.20897615708274897|
|        [green tea]|         [pancakes]|0.17251051893408134|
|         [pancakes]|        [green tea]|  0.124117053481332|
|         [pancakes]|             [milk]|0.12757201646090535|
|       